In [1]:
import torch
from transformers import AutoTokenizer
import datasets
from utils_generator import GenerationModel

from torch.utils.data import DataLoader, random_split
torch.manual_seed(9)

In [2]:
args = {
    "batch_size":16,
    "num_workers":0,
    "shuffle":True,
    "drop_last":False
    }

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")

model = GenerationModel("Helsinki-NLP/opus-mt-de-en")

/Users/gauthierroy/opt/anaconda3/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


cpu


In [4]:
data_de = datasets.load_dataset('Helsinki-NLP/tatoeba_mt','deu-eng',split=datasets.Split.TEST, 
                                verification_mode= datasets.VerificationMode.NO_CHECKS)
data_de_train, data_de_test = random_split(data_de, [0.95, 0.05])

data_fr = datasets.load_dataset('Helsinki-NLP/tatoeba_mt','eng-fra', split=datasets.Split.TEST, 
                                verification_mode= datasets.VerificationMode.NO_CHECKS)\
                                    .rename_columns({'sourceLang': 'targetlang', 'targetlang': 'sourceLang', 
                                                     'sourceString': 'targetString', 'targetString': 'sourceString'})
data_fr_train, data_fr_test = random_split(data_fr, [0.95, 0.05])

# data_nld = datasets.load_dataset('Helsinki-NLP/tatoeba_mt','eng-nld',split=datasets.Split.TEST,
#                                   verification_mode= datasets.VerificationMode.NO_CHECKS)\
#                                     .rename_columns({'sourceLang': 'targetlang', 'targetlang': 'sourceLang', 
#                                                      'sourceString': 'targetString', 'targetString': 'sourceString'})

Found cached dataset tatoeba_mt (/Users/gauthierroy/.cache/huggingface/datasets/Helsinki-NLP___tatoeba_mt/deu-eng/0.0.0/01e819f3f64a772a2ca70949061d295d3a2dc99d05183fe4776a3be23f75f619)
Found cached dataset tatoeba_mt (/Users/gauthierroy/.cache/huggingface/datasets/Helsinki-NLP___tatoeba_mt/eng-fra/0.0.0/01e819f3f64a772a2ca70949061d295d3a2dc99d05183fe4776a3be23f75f619)


In [5]:
data_de_test[0]

{'sourceLang': 'deu',
 'targetlang': 'eng',
 'sourceString': 'Kennst du die Antwort?',
 'targetString': 'Do you know the answer?'}

In [6]:
loader_de_train = DataLoader(data_de_train, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

loader_de_test = DataLoader(data_de_test, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

loader_fr_train = DataLoader(data_fr_train, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

loader_fr_test = DataLoader(data_fr_test, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

# loader_nld = DataLoader(data_nld, batch_size=args['batch_size'], num_workers=args['num_workers'], 
#                        shuffle=args['shuffle'], drop_last=args['drop_last'])

In [7]:
x_de_train = tokenizer(next(iter(loader_de_train))["sourceString"], return_tensors="pt", padding=True)
# x_de_test = tokenizer(next(iter(loader_de_test))["sourceString"], return_tensors="pt", padding=True)


# x_fr_train = tokenizer(next(iter(loader_fr_train))["sourceString"], return_tensors="pt", padding=True)
# x_fr_test = tokenizer(next(iter(loader_fr_test))["sourceString"], return_tensors="pt", padding=True)

# x_nld = tokenizer(next(iter(loader_nld))["sourceString"], return_tensors="pt", padding=True)

In [8]:
threshold = model.define_threshold(loader_fr_test, 0.95, 'Renyi', "s0", 0.1, temperature=1)

Computing anomaly score for each data:   0%|          | 0/40 [00:00<?, ?it/s]/Users/gauthierroy/opt/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Computing anomaly score for each data:  38%|███▊      | 15/40 [01:04<01:50,  4.41s/it]

In [ ]:
threshold

tensor(0.3658)

In [26]:
model.classify(x_de_train, threshold, 'Renyi', "s0", 0.1, temperature=1)

tensor(0.3658)

In [10]:
#out = model.model.generate(**x_de, return_dict_in_generate=True,output_scores=True, num_beams=1)

In [11]:
#out.scores[0].shape

In [12]:
#torch.stack(list(out.scores), dim=0).permute(1,0,2)

In [13]:
#model.softmax(torch.stack(list(out.scores), dim=0).permute(1,0,2))

In [14]:
# anomaly_score_de, sequences_de = model.anomaly_score(x_de, 'Renyi', 0.1, temperature=1)

# anomaly_score_fr, sequences_fr = model.anomaly_score(x_fr, 'Renyi', 0.1, temperature=1)

# anomaly_score_nld, sequences_nld = model.anomaly_score(x_nld, 'Renyi', 0.1, temperature=1)

In [15]:
# anomaly_score_de

In [16]:
# anomaly_score_fr

In [17]:
# anomaly_score_nld

In [18]:
# print(torch.mean(anomaly_score_de))
# print(torch.std(anomaly_score_de))


In [19]:
# print(torch.mean(anomaly_score_fr))
# print(torch.std(anomaly_score_fr))

In [20]:
# print(torch.mean(anomaly_score_nld))
# print(torch.std(anomaly_score_nld))


In [21]:
# tokenizer.batch_decode(generated_ids.sequences, skip_special_tokens=True)

In [22]:
# x = tokenizer(data_nld[0]["targetString"], return_tensors="pt")

# generated_ids = model.generate(**x)
# tokenizer.batch_decode(generated_ids, skip_special_tokens=True)